# Examples #

This notebook demonstrates usage of IBM qiskit, how to deal in this repository with circuits to be executed on the simulator, and those to be properly run on the backend ('queue').

In [ ]:
''' IMPORTS '''
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.quantum import *;
from src.thirdparty.render import *;
from src.api import *;

np.random.seed(39102901); # for repeatability

In [ ]:
'''Example usage of simulator'''
@connect_to_backend(option=BACKEND_SIMULATOR.AER)
def action(option, backend, num_shots: int):
    # create circuit:
    print('Example quantum circuit with simulator');
    u, u_inv = qk_unitary_gate_pair(theta1=np.pi/3, theta2=0, theta3=np.pi/4);
    circuit = QuantumCircuit(3, 3);
    circuit.append(u, [2]);
    circuit.barrier();
    circuit.h(1);
    circuit.x(1);
    circuit.cx(1, 2);
    circuit.x(1);
    circuit.h(1);
    circuit.cx(2, 0);
    circuit.barrier();
    circuit.append(u_inv, [2]);
    circuit.measure(0, 0);
    circuit.measure(1, 1);
    circuit.measure(2, 2);

    # display circuit:
    display(circuit.draw(output=DRAW_MODE.COLOUR.value, cregbundle=False, initial_state=True));

    # create job:
    circuit = qk_transpile(circuit, backend);
    job = qk_execute(
        experiments = circuit,
        backend = backend,
        shots = num_shots,
        optimization_level = 3,
    );
    print(latest_info(backend=backend, job=job));
    # need this to keep track internally of the latest simulated job
    latest_state.set_job(job=job, queue=False);

action(num_shots=1000);

In [ ]:
@recover_job(queue=False, ensure_job_done=True, use_latest=True)
def action(job: IBMQJob):
    result = job.result();
    counts, _ = get_counts(result, pad=True);
    display(QkVisualisation.plot_histogram(counts, title='Measurements'));

action();

In [ ]:
provider = get_ibm_account(force_reload=False);
display(provider.backends());

In [ ]:
'''Example usage of IBM backend / queue'''
@connect_to_backend(option=BACKEND.LEAST_BUSY)
def action(option, backend, num_shots: int):
    # create circuit:
    print('Example quantum circuit with backend');
    circuit = QuantumCircuit(3, 3);
    circuit.u(np.pi/3,0,np.pi/4,2);
    circuit.barrier();
    circuit.h(1);
    circuit.x(1);
    circuit.cx(1, 2);
    circuit.x(1);
    circuit.h(1);
    circuit.cx(2, 0);
    circuit.barrier();
    circuit.u(-np.pi/3,-0,-np.pi/4,2);
    circuit.measure(0, 0);
    circuit.measure(1, 1);
    circuit.measure(2, 2);

    # display circuit:
    display(circuit.draw(output=DRAW_MODE.COLOUR.value, cregbundle=False, initial_state=True));

    # create job:
    circuit = qk_transpile(circuit, backend);
    job = qk_execute(
        experiments = circuit,
        backend = backend,
        shots = num_shots,
        optimization_level = 3,
    );
    print(latest_info(backend=backend, job=job));
    # need this to keep track internally of the latest queued job
    latest_state.set_job(job=job, queue=True);

action(num_shots=100);

In [ ]:
@recover_job(queue=True, ensure_job_done=True, use_latest=True)
def action(job: IBMQJob):
    result = job.result();
    counts, _ = get_counts(result);
    display(QkVisualisation.plot_histogram(counts, title='Measurements'));

action();